In [1]:
from pib import PIB
import pandas as pd

In [2]:
pib = PIB('490dcd21-44b2-fef6-0ade-e6431a8c8fb9')

entidades = (pd.read_parquet("../../data/catalogos/estados/estados.parquet")
            #  .sort_values(by="nom_agee")
             ).nom_agee.apply(lambda x: x.capitalize())

In [3]:
def pib_real():
    df = pd.DataFrame()

    for entidad in entidades:
        print(entidad)
        temp = pib.get_data_estatal(grupo = "estado", entidad=entidad)
        temp["entidad"] = entidad
        df = pd.concat([df,temp])

    return df

df_ = pib_real()

Baja california
Baja california sur
Campeche
Colima
Guerrero
Jalisco
Nayarit
Oaxaca
Quintana roo
Sinaloa
Sonora
Veracruz de ignacio de la llave
Yucatán
Aguascalientes
Coahuila de zaragoza
Chiapas
Chihuahua
Ciudad de méxico
Durango
Guanajuato
Hidalgo
México
Michoacán de ocampo
Morelos
Nuevo león
Puebla
Querétaro
San luis potosí
Tabasco
Tamaulipas
Tlaxcala
Zacatecas


In [7]:
def pib_nom():
    df = pd.DataFrame()

    for entidad in entidades:
        print(entidad)
        temp = pib.get_data_estatal(indicador="PIB anual a precios corrientes", grupo = "estado", entidad=entidad)
        temp["entidad"] = entidad
        df = pd.concat([df,temp])

    return df

df_nom = pib_nom()

Baja california
Baja california sur
Campeche
Colima
Guerrero
Jalisco
Nayarit
Oaxaca
Quintana roo
Sinaloa
Sonora
Veracruz de ignacio de la llave
Yucatán
Aguascalientes
Coahuila de zaragoza
Chiapas
Chihuahua
Ciudad de méxico
Durango
Guanajuato
Hidalgo
México
Michoacán de ocampo
Morelos
Nuevo león
Puebla
Querétaro
San luis potosí
Tabasco
Tamaulipas
Tlaxcala
Zacatecas


In [12]:
df_.to_excel("../../data/pib/pib_realv2.xlsx")
df_nom.to_excel("../../data/pib/pib_nominalv2.xlsx")

In [3]:
df_ = pd.read_excel("../../data/pib/pib_realv2.xlsx", dtype={"periodo":str}).set_index("periodo")

df_nom = pd.read_excel("../../data/pib/pib_nominalv2.xlsx", dtype={"periodo":str}).set_index("periodo")

In [4]:
def ranking(df, sector, year:int):
    ranking = pd.DataFrame()
    year = str(year)
    for entidad in entidades:
        temp = df[df["entidad"] == entidad].loc[[year]]
        temp["entidad"] = entidad
        ranking = pd.concat([ranking,temp])
    
    ranking = ranking.reset_index(drop=True)
    ranking = ranking.set_index("entidad")
    ranking = ranking[[sector]]
    ranking["rank"] = (ranking.rank(method="first", ascending=False))
    ranking = ranking.sort_values(by="rank")

    X = ranking[sector].sum()

    ranking["% nacional"] = (ranking[sector]/X)*100

    return ranking

ranking(df_nom, "Total", 2022)

,Total,rank,% nacional
entidad,,,
Ciudad de méxico,4287768.210,1.0,14.558085
México,2647537.325,2.0,8.989076
Nuevo león,2356106.867,3.0,7.999594
Jalisco,2212640.428,4.0,7.512488
Guanajuato,1374749.364,5.0,4.667630
Veracruz de ignacio de la llave,1251659.524,6.0,4.249709
Coahuila de zaragoza,1149123.673,7.0,3.901573
Baja california,1146704.230,8.0,3.893358
Chihuahua,1125154.076,9.0,3.820190


In [5]:
def variacion(df, sector:str, year: int):
    df_f = pd.DataFrame()

    yeari = str(year - 1)
    yearf = str(year)

    for entidad in entidades:
        temp = df[df["entidad"] == entidad].loc[[yeari, yearf]]
        temp["entidad"] = entidad
        temp = temp[["entidad",sector]]
        temp["cambio %"] = (temp[sector].pct_change())*100
        # temp = temp.dropna()
        df_f = pd.concat([df_f,temp])
    

    return df_f

variacion(df_, "Total industria manufacturera", 2022)

,entidad,Total industria manufacturera,cambio %
periodo,,,
2021,Baja california,330405.218,NaN
2022,Baja california,342887.949,3.778007
2021,Baja california sur,3935.328,NaN
2022,Baja california sur,4027.591,2.344481
2021,Campeche,4221.670,NaN
...,...,...,...
2022,Tamaulipas,246568.446,6.590693
2021,Tlaxcala,40845.040,NaN
2022,Tlaxcala,42138.355,3.166394


In [6]:
def  var_nac():
    df = (variacion(df_, "Total", 2022)
     .reset_index())
    
    sum_2021 = df[df["periodo"] == "2021"]["Total"].sum()
    sum_2022 = df[df["periodo"] == "2022"]["Total"].sum()

    var = ((sum_2022 - sum_2021)/sum_2021)*100

    return var

var_nac()

3.9490002091082275

In [7]:
def participación_sectores(df, entidad, year):
    sectores = ["Total actividades primarias",
                "Total actividades secundarias",
                "Total actividades terciarias"
                ""]

    temp = (df[(df["entidad"] == entidad)]
            .loc[[str(year)]]
            .drop(columns = {"entidad"})).T
    
    total = temp.loc["Total"]
    temp["%_toal"] = (temp/total)*100
    temp = temp.loc[sectores]


    return temp

participación_sectores(df_, "Oaxaca", 2022)

periodo,2022,%_toal
Total actividades primarias,18411.766,4.521131
Total actividades secundarias,128663.403,31.594147
Total actividades terciarias,243217.449,59.723649


In [8]:
df_nom.columns

Index(['Total', 'Total actividades primarias', 'Total sector 11',
       'Agricultura ', 'Cría y explotación de animales ',
       'Pesca, caza y captura ',
       'Aprovechamiento forestal; Servicios relacionados con las actividades agropecuarias y forestales ',
       'Total actividades secundarias', 'Total Mineria', 'Mineria petrolera',
       'Mineria no petrolera',
       'Generación, transmisión, distribución y comercialización de energía eléctrica, suministro de agua y de gas natural por ductos al consumidor final',
       'Construccion', 'Total industria manufacturera',
       'Industria alimentaria', 'Industria de las bebidas y del tabaco',
       'Fabricación de insumos textiles y acabado de textiles; Fabricación de productos textiles, excepto prendas de vestir',
       'Fabricación de prendas de vestir; Curtido y acabado de cuero y piel, y fabricación de productos de cuero, piel y materiales sucedáneos',
       'Industria de la madera',
       'Industria del papel; Impresión

In [77]:
def actividades_secundarias(df, entidad):
    df = df[df["entidad"] == entidad]
    df = (df.iloc[-1:,7:13]).T
    total = df.loc["Total actividades secundarias"]

    df["% Actividades secundarias"] =  (df/total)*100
    df = df.reset_index()
    df.columns = ["Actividad","PIB 2022", "% Actividades secundarias"]
    df = df.sort_values("PIB 2022", ascending=False)

    return df

actividades_secundarias(df_, "Hidalgo")

,Actividad,PIB 2022,% Actividades secundarias
0,Total actividades secundarias,162546.508,100.000000
5,Construccion,37077.154,22.810182
4,"Generación, transmisión, distribución y comerc...",6656.535,4.095157
1,Total Mineria,5852.344,3.600412
3,Mineria no petrolera,5852.344,3.600412
2,Mineria petrolera,0.000,0.000000


In [76]:
def manufacturas(df, entidad):
    df = df[df["entidad"] == entidad]
    df = (df.iloc[-1:,13:26]).T
    total = df.loc["Total industria manufacturera"]

    df["% manufacturas"] =  (df/total)*100
    df = df.reset_index()
    df.columns = ["Actividad","PIB 2022", "% de industria manufacturera"]
    df = df.sort_values("PIB 2022", ascending=False)

    return df

manufacturas(df_, "Hidalgo")

,Actividad,PIB 2022,% de industria manufacturera
0,Total industria manufacturera,112960.475,100.000000
7,Fabricación de productos derivados del petróle...,50438.612,44.651558
1,Industria alimentaria,18134.296,16.053665
8,Fabricación de productos a base de minerales n...,11871.935,10.509813
10,Fabricación de maquinaria y equipo; Fabricació...,10477.217,9.275118
2,Industria de las bebidas y del tabaco,8388.653,7.426184
9,Industrias metálicas básicas; Fabricación de p...,4799.752,4.249054
3,Fabricación de insumos textiles y acabado de t...,2337.370,2.069193
4,Fabricación de prendas de vestir; Curtido y ac...,2336.524,2.068444
6,Industria del papel; Impresión e industrias co...,2178.325,1.928396


In [75]:
def actividades_terciarias(df, entidad):
    df = df[df["entidad"] == entidad]
    df = (df.iloc[-1:,26:-1]).T
    total = df.loc["Total actividades terciarias"]

    df["% actividades terciarias"] =  (df/total)*100
    df = df.reset_index()
    df.columns = ["Actividad","PIB 2022", "% Total actividades terciarias"]

    df = df.sort_values("PIB 2022", ascending=False)

    return df

actividades_terciarias(df_, "Hidalgo")

,Actividad,PIB 2022,% Total actividades terciarias
0,Total actividades terciarias,231447.318,100.000000
2,Comercio al por menor,47476.110,20.512707
6,Servicios inmobiliarios y de alquiler de biene...,40034.034,17.297256
3,"Transportes, correos y almacenamiento",37701.640,16.289513
1,Comercio al por mayor,37507.153,16.205482
10,Servicios educativos,18871.741,8.153796
15,"Actividades legislativas, gubernamentales, de ...",13940.691,6.023267
5,Servicios financieros y de seguros,8763.546,3.786411
11,Servicios de salud y de asistencia social,8244.637,3.562209
14,Otros servicios excepto actividades gubernamen...,6682.655,2.887333


In [39]:
class INFO_PIB:

    def __init__(self) -> None:
        pass


    def _ranking(self, df, sector, year):
        ranking = pd.DataFrame()
        year = str(year)
        for entidad in entidades:
            temp = df[df["entidad"] == entidad].loc[[year]]
            temp["entidad"] = entidad
            ranking = pd.concat([ranking,temp])
        
        ranking = ranking.reset_index(drop=True)
        ranking = ranking.set_index("entidad")
        ranking = ranking[[sector]]
        ranking["rank"] = (ranking.rank(method="first", ascending=False))
        ranking = ranking.sort_values(by="rank")

        X = ranking[sector].sum()

        ranking["% nacional"] = (ranking[sector]/X)*100

        return ranking
    

    def _variacion(self, df, sector, year):
        df_f = pd.DataFrame()
        yeari = str(year - 1)
        yearf = str(year)

        for entidad in entidades:
            temp = df[df["entidad"] == entidad].loc[[yeari, yearf]]
            temp["entidad"] = entidad
            temp = temp[["entidad",sector]]
            temp["cambio %"] = (temp[sector].pct_change())*100
            temp = temp.dropna()
            df_f = pd.concat([df_f,temp])
        

        return df_f
    
    
    def get_ranking(self,
                    df: pd.DataFrame,
                    sector: str|list = "Total", 
                    year: int = 2022,
                    estado: str = "Aguascalientes"
                    ):
        
        ranking = self._ranking(df, sector, year)
        var = ranking.loc[estado][0]
        rank = ranking.loc[estado][1]
        nac =  round(ranking.loc[estado][2],2)

        return [var, rank, nac]
    

    def get_var(self, 
                df: pd.DataFrame,
                sector: str = "Total",
                year: int = 2022,
                estado: str = "Aguascalientes",
                ):
        
        var = self._variacion(df, sector, year).set_index("entidad")
        var = round(var.loc[estado][1], 2)

        return var
    
    def actividades_secundarias(self, 
                                df: pd.DataFrame, 
                                entidad: str = "Aguascalientes"):
        df = df[df["entidad"] == entidad]
        df = (df.iloc[-1:,7:26]).T
        total = df.loc["Total actividades secundarias"]

        df =  (df/total)*100
        df.drop("Total actividades secundarias", inplace=True)
        df.columns = ["% de actividades secundarias"]

        return df
    

    def manufacturas(self,
                     df: pd.DataFrame,
                     entidad: str = "Aguascalientes"):
        
        df = df[df["entidad"] == entidad]
        df = (df.iloc[-1:,13:26]).T
        total = df.loc["Total industria manufacturera"]

        df =  (df/total)*100
        df.drop("Total industria manufacturera", inplace=True)
        df.columns = ["% de industria manufacturera"]

        return df


info = INFO_PIB()

In [45]:
def table_style(df):
    th_props = [
    ('font-size', '16px'),
    ('text-align', 'center'),
    ('font-weight', 'bold'),
    ('color', '#ffffff'),
    ('background-color', '#B38E5D')
    ]

    td_props = [
    ('font-size', '14px')
    ]

    styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
    ]

    # table
    df = (df.style
        .format(precision=0, thousands=",")
        .set_properties(**{'text-align': 'left'})
        .set_table_styles(styles))
    
    return df

In [46]:
table_style(info.manufacturas(df_, "Hidalgo"))

,% de industria manufacturera
Industria alimentaria,16
Industria de las bebidas y del tabaco,7
"Fabricación de insumos textiles y acabado de textiles; Fabricación de productos textiles, excepto prendas de vestir",2
"Fabricación de prendas de vestir; Curtido y acabado de cuero y piel, y fabricación de productos de cuero, piel y materiales sucedáneos",2
Industria de la madera,1
Industria del papel; Impresión e industrias conexas,2
Fabricación de productos derivados del petróleo y del carbón; Industria química; Industria del plástico y del hule,45
Fabricación de productos a base de minerales no metálicos,11
Industrias metálicas básicas; Fabricación de productos metálicos,4
"Fabricación de maquinaria y equipo; Fabricación de equipo de computación, comunicación, medición y de otros equipos, componentes y accesorios electrónicos; Fabricación de accesorios, aparatos eléctricos y equipo de generación",9


### Informacion Yucatan

In [50]:
df_.columns

Index(['Total', 'Total actividades primarias', 'Total sector 11',
       'Agricultura ', 'Cría y explotación de animales ',
       'Pesca, caza y captura ',
       'Aprovechamiento forestal; Servicios relacionados con las actividades agropecuarias y forestales ',
       'Total actividades secundarias', 'Total Mineria', 'Mineria petrolera',
       'Mineria no petrolera',
       'Generación, transmisión, distribución y comercialización de energía eléctrica, suministro de agua y de gas natural por ductos al consumidor final',
       'Construccion', 'Total industria manufacturera',
       'Industria alimentaria', 'Industria de las bebidas y del tabaco',
       'Fabricación de insumos textiles y acabado de textiles; Fabricación de productos textiles, excepto prendas de vestir',
       'Fabricación de prendas de vestir; Curtido y acabado de cuero y piel, y fabricación de productos de cuero, piel y materiales sucedáneos',
       'Industria de la madera',
       'Industria del papel; Impresión

In [81]:
yucatan = df_nom[df_nom["entidad"] == "Yucatán"][['Total', 'Total industria manufacturera','Fabricación de insumos textiles y acabado de textiles; Fabricación de productos textiles, excepto prendas de vestir']]
yucatan["% total"] = (yucatan.iloc[:,2]/yucatan.iloc[:,0])*100
yucatan["% manufactura"] = (yucatan.iloc[:,1]/yucatan.iloc[:,0])*100
yucatan

,Total,Total industria manufacturera,"Fabricación de insumos textiles y acabado de textiles; Fabricación de productos textiles, excepto prendas de vestir",% total,% manufactura
periodo,,,,,
2003,110603.098,11486.104,575.367,0.520209,10.384975
2004,121281.924,12634.941,558.328,0.460355,10.417827
2005,134299.364,13724.032,658.099,0.490024,10.218985
2006,146676.350,14687.119,756.183,0.515545,10.013284
2007,160017.756,15799.787,706.739,0.441663,9.873771
2008,171500.851,17210.866,655.879,0.382435,10.035441
2009,181462.700,18861.209,638.591,0.351913,10.393987
2010,193314.432,19673.707,686.200,0.354966,10.177050
2011,205197.013,21220.129,664.714,0.323939,10.341344


In [80]:
yucatan = df_[df_["entidad"] == "Yucatán"][['Total', 'Total industria manufacturera','Fabricación de insumos textiles y acabado de textiles; Fabricación de productos textiles, excepto prendas de vestir']]
yucatan.pct_change()*100

,Total,Total industria manufacturera,"Fabricación de insumos textiles y acabado de textiles; Fabricación de productos textiles, excepto prendas de vestir"
periodo,,,
2003,NaN,NaN,NaN
2004,3.958834,4.399751,-7.024653
2005,4.613039,2.604196,8.470555
2006,4.749661,5.272536,10.584066
2007,2.246570,1.099466,-8.844860
2008,1.562346,0.158899,-12.893660
2009,-2.883971,-0.863418,-12.193213
2010,3.717752,1.885309,4.085621
2011,2.200261,3.207759,-3.777321
